<a href="https://colab.research.google.com/github/geoffwoollard/learn_cryoem_math/blob/master/nb/healpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install healpy

     |████████████████████████████████| 23.6 MB 18.0 MB/s 


In [13]:
!pip install pytorch3d

     |████████████████████████████████| 30.0 MB 1.5 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220305-py3-none-any.whl size=61214 sha256=d8dedac5597b7d9ce3e13959939034d5b80cc923d708cae2dadf4ef90482fe67
  Stored in directory: /root/.cache/pip/wheels/b5/b7/6e/43b1693d06fac3633af48db68557513b0a37ab38b0a8b798f9
Successfully built fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [32]:
import numpy as np
import healpy as hp
import pytorch3d.transforms
import torch
from torch import tensor

# Resources
* https://healpy.readthedocs.io/en/latest/tutorial.html#NSIDE-and-ordering

# Eulers (theta, phi) to rotations

In [17]:
NSIDE = 1
NPIX = hp.nside2npix(NSIDE)
print(NPIX)

12


In [36]:
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))


In [37]:
np.rad2deg(theta)

array([ 48.1896851,  48.1896851,  48.1896851,  48.1896851,  90.       ,
        90.       ,  90.       ,  90.       , 131.8103149, 131.8103149,
       131.8103149, 131.8103149])

In [38]:
np.rad2deg(phi)

array([ 45., 135., 225., 315.,   0.,  90., 180., 270.,  45., 135., 225.,
       315.])

In [68]:
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX) # arange?
n_repeats = 2
psi.repeat(1,n_repeats).T

tensor([[0.0000],
        [0.5236],
        [1.0472],
        [1.5708],
        [2.0944],
        [2.6180],
        [3.1416],
        [3.6652],
        [4.1888],
        [4.7124],
        [5.2360],
        [5.7596],
        [0.0000],
        [0.5236],
        [1.0472],
        [1.5708],
        [2.0944],
        [2.6180],
        [3.1416],
        [3.6652],
        [4.1888],
        [4.7124],
        [5.2360],
        [5.7596]])

In [69]:
euler_angles = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles.repeat(n_repeats,1)


tensor([[0.8411, 0.7854],
        [0.8411, 2.3562],
        [0.8411, 3.9270],
        [0.8411, 5.4978],
        [1.5708, 0.0000],
        [1.5708, 1.5708],
        [1.5708, 3.1416],
        [1.5708, 4.7124],
        [2.3005, 0.7854],
        [2.3005, 2.3562],
        [2.3005, 3.9270],
        [2.3005, 5.4978],
        [0.8411, 0.7854],
        [0.8411, 2.3562],
        [0.8411, 3.9270],
        [0.8411, 5.4978],
        [1.5708, 0.0000],
        [1.5708, 1.5708],
        [1.5708, 3.1416],
        [1.5708, 4.7124],
        [2.3005, 0.7854],
        [2.3005, 2.3562],
        [2.3005, 3.9270],
        [2.3005, 5.4978]], dtype=torch.float64)

In [71]:
n_repeats = NPIX
euler_angles = torch.hstack([euler_angles.repeat(n_repeats,1), psi.repeat(1,n_repeats).T])

In [74]:
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='XYZ')
rots.shape

torch.Size([144, 3, 3])